In [1]:
import numpy as np
import pandas as pd

from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import locale
import os
import xlrd as xlrd

In [2]:
# Retrieve current working directory (`cwd`)
cwd = os.getcwd()
cwd

'/Users/somya/Projects/nyc_property_sales'

In [3]:

# Change directory 
#os.chdir("data/raw")

# List all files and directories in current directory
excel_files = os.listdir('data/raw')
excel_files

['2015_manhattan.xls',
 '2011_manhattan.xls',
 '2016_manhattan.xls',
 '.DS_Store',
 '2012_manhattan.xls',
 'rollingsales_manhattan.xls',
 'sales_manhattan_03.xls',
 'sales_manhattan_06.xls',
 'sales_manhattan_04.xls',
 'sales_2007_manhattan.xls',
 'sales_manhattan_05.xls',
 '2009_manhattan.xls',
 '2013_manhattan.xls',
 '2017_manhattan.xls',
 'sales_2008_manhattan.xls',
 '2010_manhattan.xls',
 '2014_manhattan.xls']

In [15]:
# Constants
raw_directory = "data/raw/"
csv_directory = "data/csv/"

# Only handle xls files

excel_files = [k for k in excel_files if '.xls' in k]

In [17]:
excel_files[:len(excel_files)]

['2015_manhattan.xls',
 '2011_manhattan.xls',
 '2016_manhattan.xls',
 '2012_manhattan.xls',
 'rollingsales_manhattan.xls',
 'sales_manhattan_03.xls',
 'sales_manhattan_06.xls',
 'sales_manhattan_04.xls',
 'sales_2007_manhattan.xls',
 'sales_manhattan_05.xls',
 '2009_manhattan.xls',
 '2013_manhattan.xls',
 '2017_manhattan.xls',
 'sales_2008_manhattan.xls',
 '2010_manhattan.xls',
 '2014_manhattan.xls']

In [18]:

# Load the first file
all_sales_data = pd.DataFrame()

for excel_file in excel_files:
    print(excel_file)
    # Read excel, Note the headers coulb in row 4 or row 5 (index=3 or 4). 
    yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=3, encoding='sys.getfilesystemencoding()')
    print(yearly_sales_data.shape)
    # Check for start of data if the first column is a "BOROUGH"
    if not yearly_sales_data.columns[0].startswith('BOROUGH'):
        # Otherwise the data starts from row 5.
         yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=4, encoding='sys.getfilesystemencoding()')
            
    yearly_sales_data.rename(columns=lambda x: x.strip(), inplace=True)
        
    all_sales_data = all_sales_data.append(yearly_sales_data)
    print(all_sales_data.shape)

    
all_sales_data




2015_manhattan.xls
(24990, 21)
(24989, 21)
2011_manhattan.xls
(21501, 21)
(46489, 21)
2016_manhattan.xls
(21242, 21)
(67730, 21)
2012_manhattan.xls
(26259, 21)
(93988, 21)
rollingsales_manhattan.xls
(16829, 21)
(110816, 21)
sales_manhattan_03.xls
(22210, 21)
(133026, 21)
sales_manhattan_06.xls
(26352, 21)
(159378, 21)
sales_manhattan_04.xls
(25894, 21)
(185272, 21)
sales_2007_manhattan.xls
(28439, 21)
(213711, 21)
sales_manhattan_05.xls
(26388, 21)
(240099, 21)
2009_manhattan.xls
(19166, 21)
(259265, 21)
2013_manhattan.xls
(26716, 21)
(285980, 21)
2017_manhattan.xls
(18643, 21)
(304622, 23)
sales_2008_manhattan.xls
(25994, 21)
(330616, 23)
2010_manhattan.xls
(17296, 21)
(347912, 23)
2014_manhattan.xls
(24525, 21)
(372436, 23)


,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AS OF FINAL ROLL 17/18,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,...,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,SALE PRICE,TAX CLASS AS OF FINAL ROLL 17/18,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
0,526 EAST 5TH STREET,,400,1,NaN,A4,A4,01 ONE FAMILY DWELLINGS,0,,...,ALPHABET CITY,1,2015-03-18,6500000,NaN,1,1,1,1900,10009
1,264 EAST 7TH STREET,,376,1,NaN,C0,C0,03 THREE FAMILY DWELLINGS,0,,...,ALPHABET CITY,3,2015-10-22,3775000,NaN,1,1,3,1900,10009
2,243 EAST 7TH STREET,,377,1,NaN,C0,C0,03 THREE FAMILY DWELLINGS,0,,...,ALPHABET CITY,3,2015-06-24,2900000,NaN,1,1,3,1899,10009
3,238 EAST 4TH STREET,-,399,1,NaN,R6,R6,04 TAX CLASS 1 CONDOS,0,,...,ALPHABET CITY,1,2015-06-15,6995000,NaN,1C,1,1,1955,10009
4,326 EAST 4TH STREET,,373,1,NaN,C1,C1,07 RENTALS - WALKUP APARTMENTS,0,,...,ALPHABET CITY,10,2015-11-17,20000000,NaN,2B,2,10,1899,10009
5,328 EAST 4TH STREET,,373,1,NaN,C1,C1,07 RENTALS - WALKUP APARTMENTS,0,,...,ALPHABET CITY,10,2015-11-17,0,NaN,2B,2,10,1900,10009
6,102 AVENUE C,,376,1,NaN,C4,C4,07 RENTALS - WALKUP APARTMENTS,3,,...,ALPHABET CITY,26,2015-02-05,14550000,NaN,2,2,29,1900,10009
7,102 AVENUE C,,376,1,NaN,C4,C4,07 RENTALS - WALKUP APARTMENTS,3,,...,ALPHABET CITY,26,2015-02-05,0,NaN,2,2,29,1900,10009
8,112 AVENUE C,,377,1,NaN,C7,C7,07 RENTALS - WALKUP APARTMENTS,3,,...,ALPHABET CITY,22,2015-09-10,16700000,NaN,2,2,25,1900,10009
9,112 AVENUE C,,377,1,NaN,C7,C7,07 RENTALS - WALKUP APARTMENTS,3,,...,ALPHABET CITY,22,2015-09-10,0,NaN,2,2,25,1900,10009


In [114]:
all_sales_data.sample(10)


,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AS OF FINAL ROLL 17/18,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,...,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,SALE PRICE,TAX CLASS AS OF FINAL ROLL 17/18,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
12572,102 WEST 57TH STREET,,1009,1,NaN,H2,H2,25 LUXURY HOTELS,2,,...,MIDTOWN WEST,0,2013-01-14,48835,NaN,4,4,2,2007,10019
24804,"215 WEST 91ST STREET, 33",,1239,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER WEST SIDE (79-96),0,2012-02-22,1162500,NaN,2,2,0,1917,10025
13856,1335 AVENUE OF THE AMERIC,HU2,1006,1,NaN,RH,RH,45 CONDO HOTELS,0,,...,MIDTOWN WEST,0,2015-10-28,45000,NaN,4,4,1,1963,10019
7045,"45 PARK TERRACE WEST, 4F",,2243,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,INWOOD,0,2005-06-27,245000,NaN,2,2,0,1940,10034
4261,"92 HORATIO STREET, 3C",,642,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,GREENWICH VILLAGE-WEST,0,2011-06-29,315000,NaN,2,2,0,1920,10014
17556,"80 WARREN STREET, 67",,137,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,TRIBECA,0,2013-07-30,2850000,NaN,2,2,0,1915,10007
24118,"321 WEST 90 STREET, 3E",,1251,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER WEST SIDE (79-96),0,2004-08-18,540000,NaN,2,2,0,1929,10024
4954,"1 WEST 126TH STREET, 6F",,1724,1,D4,NaN,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,HARLEM-CENTRAL,0,2017-05-09,302000,2,NaN,2,0,1940,10035
10435,350 WEST 42ND STREET,20D,1032,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,MIDTOWN WEST,1,2014-08-26,1100000,NaN,2,2,1,2004,10036
22043,"161 WEST 75TH STREET, 11C",,1147,1,NaN,R9,R9,17 CONDO COOPS,0,,...,UPPER WEST SIDE (59-79),0,2014-07-25,1250000,NaN,2,2,0,1925,10023


In [19]:
# save to csv
all_sales_data.to_csv(csv_directory+"manhattan.csv")